In [884]:
import pandas as pd
import numpy as np
import scipy.sparse as scp

In [885]:
pip install catboost

### Открытие файлов

In [886]:
all_users = pd.read_csv('/content/drive/MyDrive/iVision_contest/01_all_users.csv')

In [887]:
all_users.head()

,contract_id,day_or_month_contract
0,29516,0
1,27462,0
2,21218,0
3,4869,0
4,33846,0


In [888]:
all_users.contract_id.min(), all_users.contract_id.max(), len(pd.unique(all_users.contract_id))

(1002, 274773, 6800)

In [889]:
events_log = pd.read_csv('/content/drive/MyDrive/iVision_contest/02_events_log.csv')
events_log.head()

,contract_id,event_date,event_type
0,29516,2021-03-28 16:24:30,Добавление в Обращались с номеров
1,29516,2021-03-28 16:27:41,Обращение в службу заботы о клиентах
2,29516,2021-03-28 16:29:56,Выключение IPTV-пакета
3,29516,2021-03-28 16:29:56,Включение IPTV-пакета
4,29516,2021-03-28 16:31:17,"Включение услуги ""Ограничение функций ТВ"""


In [890]:
user_events_log = pd.read_csv('/content/drive/MyDrive/iVision_contest/03_lk_events_log.csv')
user_events_log.head()

,contract_id,event_date,name
0,33846,2021-03-19 15:12:52,Информер ВК. Показ
1,33846,2021-03-27 18:46:04,Информер ВК. Показ
2,33846,2021-04-03 18:55:22,Информер ВК. Показ
3,22754,2021-03-09 19:40:53,Включение интернета на 20 минут
4,22754,2021-03-09 19:40:53,Включение интернета на 20 минут


In [891]:
is_blocked = pd.read_csv('/content/drive/MyDrive/iVision_contest/04_is_blocked.csv')
is_blocked.head()

,contract_id,blocked
0,29516,1
1,27462,1
2,21218,1
3,4869,1
4,33846,1


In [892]:
test_df = pd.read_csv('/content/drive/MyDrive/iVision_contest/test.csv')
test_df.head()

,contract_id,blocked
0,1034,NaN
1,1074,NaN
2,1145,NaN
3,1261,NaN
4,1315,NaN


In [893]:
len(test_df.contract_id.unique()), len(test_df.contract_id)

(300, 301)

id 2999 повторяеться два раза

In [894]:
z = set(all_users.contract_id.unique())
p1 = set(events_log.contract_id.unique())
p2 = set(user_events_log.contract_id.unique())
p3 = set(is_blocked.contract_id.unique())
t = set(test_df.contract_id.unique())

In [895]:
p4 = p1 | p2  # все id пользователей у которых есть хоть какая то инфа
p5 = p4 & p3  # train
p6 = p4 & t  # test
print(len(p6))

205


In [896]:
test_id = list(test_df.contract_id.unique())
train_id = list(is_blocked.contract_id.unique())
print(len(test_id), len(train_id))

300 6501


### Создание словарей ind -> original_id для дальнейшей работы

In [897]:
wr_event_to_idx = {event_id : idx for idx, event_id in enumerate(events_log.event_type.unique())}
idx_to_wr_event = {i: event for event, i in wr_event_to_idx.items()}

event_to_idx = {event_id : idx for idx, event_id in enumerate(user_events_log.name.unique())}
idx_to_event = {i: event for event, i in event_to_idx.items()}

In [898]:
user_to_idx = {event_id : idx for idx, event_id in enumerate(train_id)}
idx_to_user = {i: event for event, i in user_to_idx.items()}

In [899]:
test_events_log = events_log.loc[events_log['contract_id'].isin(train_id)].copy()
test_events_log['contract_id'] = test_events_log['contract_id'].apply(lambda x: user_to_idx[x])
test_events_log['event_type'] = test_events_log['event_type'].apply(lambda x: wr_event_to_idx[x])
test_events_log.head()

,contract_id,event_date,event_type
0,0,2021-03-28 16:24:30,0
1,0,2021-03-28 16:27:41,1
2,0,2021-03-28 16:29:56,2
3,0,2021-03-28 16:29:56,3
4,0,2021-03-28 16:31:17,4


In [900]:
test_user_events_log = user_events_log.loc[user_events_log['contract_id'].isin(train_id)].copy()
test_user_events_log['contract_id'] = test_user_events_log['contract_id'].apply(lambda x: user_to_idx[x])
test_user_events_log['name'] = test_user_events_log['name'].apply(lambda x: event_to_idx[x])
test_user_events_log.head()

,contract_id,event_date,name
0,4,2021-03-19 15:12:52,0
1,4,2021-03-27 18:46:04,0
2,4,2021-04-03 18:55:22,0
3,6,2021-03-09 19:40:53,1
4,6,2021-03-09 19:40:53,1


In [901]:
test_events_log['const'] = np.ones(test_events_log.shape[0])
test_user_events_log['const'] = np.ones(test_user_events_log.shape[0])
test_events_log.head()

,contract_id,event_date,event_type,const
0,0,2021-03-28 16:24:30,0,1.0
1,0,2021-03-28 16:27:41,1,1.0
2,0,2021-03-28 16:29:56,2,1.0
3,0,2021-03-28 16:29:56,3,1.0
4,0,2021-03-28 16:31:17,4,1.0


### Работа с первой таблицей

Преобразовывыаем нашу 2-ую таблицу в TfidfVectorizer и CountVectorizer

In [902]:
matrica_1 = scp.coo_matrix(
        (
            test_events_log['const'],  # оценки пользователей
            (test_events_log['contract_id'], test_events_log['event_type'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(train_id), len(wr_event_to_idx))  # размеры матрицы рейтингов
    ).todense().A

In [903]:
work_data = pd.DataFrame(data=matrica_1, columns=list(wr_event_to_idx.keys()))
work_data['contract_id'] = [idx_to_user[i] for i in range(len(train_id))]
work_data.head()

,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета,Включение IPTV-пакета,"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена тарифа,Удаление из Обращались с номеров,Смена пароля,Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут,Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение,Отключение скидки,Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж,Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение,Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,contract_id
0,1.0,5.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29516
1,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27462
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21218
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4869
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33846


In [904]:
work_data = pd.merge(work_data, is_blocked, on='contract_id').drop_duplicates()  # only countvec

**Tf-idf**

In [905]:
st = ' '.join(["d"+str(i) for i in test_events_log.event_type.unique()])

In [906]:
test_events_log
unic = set()
res = [st]
for i in range(len(train_id)):
  user = test_events_log[test_events_log.contract_id == i]
  events = ' '.join(["d"+str(i) for i in list(sorted(user.event_type))])
  if len(events) == 0:
    events = '999'
  res.append(events)
  for i in events.split():
    unic.add(i)

In [907]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_1 = TfidfVectorizer()
vectorizer_1 = vectorizer_1.fit(res)   # сделать просто fit
X = vectorizer_1.transform(res)
print(vectorizer_1.get_feature_names_out())

['999' 'd0' 'd1' 'd10' 'd11' 'd12' 'd13' 'd14' 'd15' 'd16' 'd17' 'd18'
 'd19' 'd2' 'd20' 'd21' 'd22' 'd23' 'd24' 'd25' 'd26' 'd27' 'd28' 'd29'
 'd3' 'd30' 'd31' 'd32' 'd33' 'd34' 'd35' 'd36' 'd37' 'd38' 'd39' 'd4'
 'd40' 'd41' 'd42' 'd43' 'd44' 'd45' 'd46' 'd47' 'd48' 'd49' 'd5' 'd50'
 'd51' 'd52' 'd53' 'd54' 'd55' 'd6' 'd60' 'd61' 'd62' 'd63' 'd64' 'd65'
 'd66' 'd67' 'd68' 'd69' 'd7' 'd70' 'd71' 'd72' 'd73' 'd8' 'd9']


In [908]:
zz = pd.DataFrame(X.toarray()[1:], columns=vectorizer_1.get_feature_names_out())
zz.head()

,999,d0,d1,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d2,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d3,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d4,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d5,d50,d51,d52,d53,d54,d55,d6,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d7,d70,d71,d72,d73,d8,d9
0,0.0,0.189542,0.534815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.262126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.234494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.430923,0.0,0.0,0.0,0.0,0.0,0.0,0.386506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206568,0.0,0.0,0.0,0.0,0.289797,0.000000
1,0.0,0.367819,0.622706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400858,0.0,0.0,0.0,0.0,0.562369,0.000000
2,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.000000,0.440463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.897771
4,0.0,0.547854,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.622772,0.0,0.0,0.0,0.0,0.0,0.0,0.558579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [909]:
zz['contract_id'] = [idx_to_user[i] for i in range(len(train_id))]

In [910]:
zz_2 = pd.merge(zz, is_blocked, on='contract_id').drop_duplicates()

In [911]:
zz_2.head() # only tf-idf

,999,d0,d1,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d2,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d3,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d4,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d5,d50,d51,d52,d53,d54,d55,d6,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d7,d70,d71,d72,d73,d8,d9,contract_id,blocked
0,0.0,0.189542,0.534815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.262126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.234494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.430923,0.0,0.0,0.0,0.0,0.0,0.0,0.386506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206568,0.0,0.0,0.0,0.0,0.289797,0.000000,29516,1
1,0.0,0.367819,0.622706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400858,0.0,0.0,0.0,0.0,0.562369,0.000000,27462,1
2,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,21218,1
3,0.0,0.000000,0.440463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.897771,4869,1
4,0.0,0.547854,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.622772,0.0,0.0,0.0,0.0,0.0,0.0,0.558579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,33846,1


Группировка CountVectorise and tfidf

In [912]:
train_work_df = pd.merge(work_data, zz, on='contract_id') 

In [913]:
train_work_df.head()

,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета,Включение IPTV-пакета,"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена тарифа,Удаление из Обращались с номеров,Смена пароля,Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут,Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",...,d36,d37,d38,d39,d4,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d5,d50,d51,d52,d53,d54,d55,d6,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d7,d70,d71,d72,d73,d8,d9
0,1.0,5.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.304327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.430923,0.0,0.0,0.0,0.0,0.0,0.0,0.386506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206568,0.0,0.0,0.0,0.0,0.289797,0.000000
1,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400858,0.0,0.0,0.0,0.0,0.562369,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.897771
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.622772,0.0,0.0,0.0,0.0,0.0,0.0,0.558579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


### Работа со второй таблицей

Преобразовывыаем нашу 3-ую таблицу в TfidfVectorizer и CountVectorizer

In [914]:
matrica_2 = scp.coo_matrix(
        (
            test_user_events_log['const'],  # оценки пользователей
            (test_user_events_log['contract_id'], test_user_events_log['name'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(train_id), len(event_to_idx))  # размеры матрицы рейтингов
    ).todense().A

In [915]:
user_data = pd.DataFrame(data=matrica_2, columns=list(event_to_idx.keys()))
user_data['contract_id'] = [idx_to_user[i] for i in range(len(train_id))]
user_data.head()

,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,contract_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29516
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27462
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21218
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4869
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33846


In [916]:
user_data = pd.merge(user_data, is_blocked, on='contract_id').drop_duplicates()
user_data.head()

,Информер ВК. Показ,Включение интернета на 20 минут,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение,Принято заявление и доп. соглашение по переезду,Смена пароля,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета,Выключение IPTV-пакета,Смена тарифа,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,Закрытие информатора об изменении уведомлений через ВК платформу,Включение управления услугами по паролю,Общий счет. Отсоединение реципиента,Общий счет. Отсоединение от донора,"Отключение услуги ""Стоп-тариф""",Подписка на ivi. Включение триала,Подписка на ivi. Отключение автопродления подписки,Отключение скидки,contract_id,blocked
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29516,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27462,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21218,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4869,1
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33846,1


**Группировка CountVectorise and tfidf**

In [917]:
st = ' '.join(["u"+str(i) for i in test_user_events_log.name.unique()])

In [918]:
unic = set()
res = [st]
for i in range(len(train_id)):
  user = test_user_events_log[test_user_events_log.contract_id == i]
  events = ' '.join(["u"+str(i) for i in list(sorted(user.name))])
  if len(events) == 0:
    events = '111'
  res.append(events)
  for i in events.split():
    unic.add(i)


In [919]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_2 = TfidfVectorizer()
vectorizer_2 = vectorizer_2.fit(res)
X = vectorizer_2.transform(res)   # сделать просто fit
print(vectorizer_2.get_feature_names_out())

['111' 'u0' 'u1' 'u10' 'u11' 'u12' 'u13' 'u14' 'u15' 'u16' 'u17' 'u18'
 'u19' 'u2' 'u20' 'u21' 'u22' 'u23' 'u24' 'u25' 'u26' 'u27' 'u28' 'u29'
 'u3' 'u30' 'u31' 'u32' 'u33' 'u34' 'u35' 'u36' 'u37' 'u38' 'u39' 'u4'
 'u40' 'u41' 'u42' 'u43' 'u44' 'u45' 'u46' 'u47' 'u5' 'u6' 'u7' 'u8' 'u9']


In [920]:
zz = pd.DataFrame(X.toarray()[1:], columns=vectorizer_2.get_feature_names_out())
zz['contract_id'] = [idx_to_user[i] for i in range(len(train_id))]
zz.head()  # only tfidf

,111,u0,u1,u10,u11,u12,u13,u14,u15,u16,u17,u18,u19,u2,u20,u21,u22,u23,u24,u25,u26,u27,u28,u29,u3,u30,u31,u32,u33,u34,u35,u36,u37,u38,u39,u4,u40,u41,u42,u43,u44,u45,u46,u47,u5,u6,u7,u8,u9,contract_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29516
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27462
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21218
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4869
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33846


### Создание фичей

In [921]:
work_data_2 = work_data.copy().drop(['blocked'], axis=1)

In [926]:
is_ampty_work = np.zeros(len(train_id))
is_ampty_user = np.zeros(len(train_id))
count_call_work = np.zeros(len(train_id))
count_call_user = np.zeros(len(train_id))
for i in range(len(train_id)):
  # есть ли какая то инфа
  if idx_to_user[i] not in p1:
    is_ampty_work[i] = 1
  else:
    qq = test_events_log[test_events_log.contract_id == i]
    count_call_work[i] = qq.shape[0]
  if idx_to_user[i] not in p2:
    is_ampty_user[i] = 1
  else:
    qq = test_user_events_log[test_user_events_log.contract_id == i]
    count_call_user[i] = qq.shape[0]
  
print(is_ampty_work.mean())
print(is_ampty_user.mean())

0.7352714966928164
0.45239193970158437


In [927]:
work_data_2['is_ampty_work'] = is_ampty_work
work_data_2['count_call_work'] = count_call_work
user_data['is_ampty_user'] = is_ampty_user
user_data['count_call_user'] = count_call_user
work_data_2.head()

,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета,Включение IPTV-пакета,"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена тарифа,Удаление из Обращались с номеров,Смена пароля,Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут,Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение,Отключение скидки,Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж,Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение,Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,contract_id,is_ampty_work,count_call_work
0,1.0,5.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29516,0.0,15.0
1,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27462,0.0,6.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21218,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4869,0.0,2.0
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33846,0.0,3.0


In [928]:
const = pd.Timestamp('2021-03-01 00:06:34')
const_2 = pd.Timestamp('2021-03-01 00:02:01')
def find_out_the_time_work(id):
  qq = test_events_log[test_events_log.contract_id == id]
  all_time = []
  for i in qq.event_date:
    time = pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S.%f')
    #print(time)
    all_time.append(time)
  mi = (min(all_time)-const).days
  ma = (max(all_time)-const).days
  dif = ma-mi
  a = [i.days * 86400 +
       i.seconds for i in np.diff(np.array(all_time))]
  #print(sum(a) // len(all_time))
  return (min(all_time)-const).days, (max(all_time)-const).days, dif, sum(a) // len(all_time)

def find_out_the_time_user(id):
  qq = test_user_events_log[test_user_events_log.contract_id == id]
  all_time = []
  for i in qq.event_date:
    time = pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S.%f')
    #print(time)
    all_time.append(time)
  mi = (min(all_time)-const_2).days
  ma = (max(all_time)-const_2).days
  dif = ma-mi
  a = [i.days * 86400 +
       i.seconds for i in np.diff(np.array(all_time))]
  #print(sum(a) // len(all_time))
  return (min(all_time)-const_2).days, (max(all_time)-const_2).days, dif, int(sum(a) // len(all_time))

In [929]:
first_event_work = np.zeros(len(train_id))  # первое событие
first_event_user = np.zeros(len(train_id))
last_event_work = np.zeros(len(train_id))  # последнее событие
last_event_user = np.zeros(len(train_id))
difference_events_work = np.zeros(len(train_id))  # разность первого и последнего
difference_events_user = np.zeros(len(train_id))
mean_events_work = np.zeros(len(train_id))  # среднее разность всех событий
mean_events_user = np.zeros(len(train_id))
for i in range(len(train_id)):
  # есть ли какая то инфа
  if idx_to_user[i] in p1:
    first_event_work[i], last_event_work[i], difference_events_work[i], mean_events_work[i] = find_out_the_time_work(i)
  if idx_to_user[i] in p2:
    first_event_user[i], last_event_user[i], difference_events_user[i], mean_events_user[i] = find_out_the_time_user(i)

## Соединение всех фичей

In [930]:
big_df_last1 = pd.merge(user_data, work_data_2, on='contract_id')
big_df_last1.head()

,Информер ВК. Показ,Включение интернета на 20 минут_x,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж_x,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение_x,Принято заявление и доп. соглашение по переезду,Смена пароля_x,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета_x,Выключение IPTV-пакета_x,Смена тарифа_x,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение_x,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,...,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение_y,Отключение скидки_y,Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж_y,Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение_y,Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,is_ampty_work,count_call_work
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [931]:
big_df_last1 = pd.merge(big_df_last1, all_users, on='contract_id').drop_duplicates()

In [932]:
big_df_last1['first_event_work'] = first_event_work
big_df_last1['first_event_user'] = first_event_user
big_df_last1['last_event_work'] = last_event_work
big_df_last1['last_event_user'] = last_event_user
big_df_last1['difference_events_work'] = difference_events_work
big_df_last1['difference_events_user'] = difference_events_user
big_df_last1['mean_events_work'] = mean_events_work
big_df_last1['mean_events_user'] = mean_events_user
big_df_last1.head()

,Информер ВК. Показ,Включение интернета на 20 минут_x,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж_x,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение_x,Принято заявление и доп. соглашение по переезду,Смена пароля_x,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета_x,Выключение IPTV-пакета_x,Смена тарифа_x,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение_x,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,...,Удаление телефона из записи,Без границ бесплатно. Включение_y,Отключение скидки_y,Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж_y,Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение_y,Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,is_ampty_work,count_call_work,day_or_month_contract,first_event_work,first_event_user,last_event_work,last_event_user,difference_events_work,difference_events_user,mean_events_work,mean_events_user
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0,27.0,0.0,37.0,0.0,10.0,0.0,57413.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0,35.0,0.0,37.0,0.0,2.0,0.0,29743.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,17.0,0.0,17.0,0.0,0.0,0.0,18.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [933]:
big_df_last1.shape

(6501, 137)

### tfidf

In [934]:
big_df_last2 = pd.merge(zz_2, zz, on='contract_id')
big_df_last2.head()

,999,d0,d1,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d2,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d3,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d4,d40,d41,d42,d43,...,u16,u17,u18,u19,u2,u20,u21,u22,u23,u24,u25,u26,u27,u28,u29,u3,u30,u31,u32,u33,u34,u35,u36,u37,u38,u39,u4,u40,u41,u42,u43,u44,u45,u46,u47,u5,u6,u7,u8,u9
0,0.0,0.189542,0.534815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.262126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.234494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304327,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.367819,0.622706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.440463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.547854,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [935]:
big_df_last2.shape

(6501, 122)

In [936]:
big_df_last = pd.merge(big_df_last1, big_df_last2, on='contract_id')
big_df_last.shape

(6501, 258)

In [937]:
big_df_last = big_df_last.drop(['blocked_y'], axis=1)

In [938]:
big_df_last = big_df_last.drop(["contract_id"], axis=1).drop_duplicates()


In [939]:
big_df_last.shape

(3912, 256)

## Обучение модели

In [940]:
from sklearn.model_selection import train_test_split

X_user, y_user = big_df_last.drop(['blocked_x'], axis=1), big_df_last['blocked_x']
X_train_user, X_test_user, y_train_user, y_test_user = train_test_split(X_user, y_user, 
                                                    train_size=0.9, 
                                                    random_state=42)

from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_user)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_user)
class_weights = dict(zip(classes, weights))

model = CatBoostClassifier(verbose=1,
                               custom_loss=['AUC'], eval_metric='AUC',
                               class_weights=class_weights, iterations = 130, depth=6)

model.fit(X_train_user, y_train_user, eval_set=(X_test_user, y_test_user))
preds_proba = model.predict(X_test_user)
lr_auc_test = roc_auc_score(y_test_user, preds_proba)
print(lr_auc_test)

Learning rate set to 0.105036
0:	test: 0.6244011	best: 0.6244011 (0)	total: 14.3ms	remaining: 1.84s
1:	test: 0.6698092	best: 0.6698092 (1)	total: 24.1ms	remaining: 1.54s
2:	test: 0.6748891	best: 0.6748891 (2)	total: 33.9ms	remaining: 1.43s
3:	test: 0.6854037	best: 0.6854037 (3)	total: 43.5ms	remaining: 1.37s
4:	test: 0.6823203	best: 0.6854037 (3)	total: 53.5ms	remaining: 1.34s
5:	test: 0.6920586	best: 0.6920586 (5)	total: 63.1ms	remaining: 1.3s
6:	test: 0.6946539	best: 0.6946539 (6)	total: 72.7ms	remaining: 1.28s
7:	test: 0.7105368	best: 0.7105368 (7)	total: 85.9ms	remaining: 1.31s
8:	test: 0.7090062	best: 0.7105368 (7)	total: 95.5ms	remaining: 1.28s
9:	test: 0.7204525	best: 0.7204525 (9)	total: 105ms	remaining: 1.26s
10:	test: 0.7257986	best: 0.7257986 (10)	total: 115ms	remaining: 1.24s
11:	test: 0.7169920	best: 0.7257986 (10)	total: 124ms	remaining: 1.22s
12:	test: 0.7177019	best: 0.7257986 (10)	total: 150ms	remaining: 1.35s
13:	test: 0.7182564	best: 0.7257986 (10)	total: 165ms	remai

## Формирование ответа

In [941]:
user_to_idx = {event_id : idx for idx, event_id in enumerate(test_id)}
idx_to_user = {i: event for event, i in user_to_idx.items()}

test_events_log = events_log.loc[events_log['contract_id'].isin(test_id)].copy()
test_events_log['contract_id'] = test_events_log['contract_id'].apply(lambda x: user_to_idx[x])
test_events_log['event_type'] = test_events_log['event_type'].apply(lambda x: wr_event_to_idx[x])

test_user_events_log = user_events_log.loc[user_events_log['contract_id'].isin(test_id)].copy()
test_user_events_log['contract_id'] = test_user_events_log['contract_id'].apply(lambda x: user_to_idx[x])
test_user_events_log['name'] = test_user_events_log['name'].apply(lambda x: event_to_idx[x])

test_events_log['const'] = np.ones(test_events_log.shape[0])
test_user_events_log['const'] = np.ones(test_user_events_log.shape[0])

matrica_1 = scp.coo_matrix(
        (
            test_events_log['const'],  # оценки пользователей
            (test_events_log['contract_id'], test_events_log['event_type'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(test_id), len(wr_event_to_idx))  # размеры матрицы рейтингов
    ).todense().A

work_data = pd.DataFrame(data=matrica_1, columns=list(wr_event_to_idx.keys()))
work_data['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]

In [942]:
train_id
unic = set()
st = ' '.join(["d"+str(i) for i in test_events_log.event_type.unique()])
res = [st]
for i in range(len(test_id)):
  user = test_events_log[test_events_log.contract_id == i]
  events = ' '.join(["d"+str(i) for i in list(sorted(user.event_type))])
  if len(events) == 0:
    events = '999'
  res.append(events)
  for i in events.split():
    unic.add(i)
X = vectorizer_1.transform(res)   # сделать просто fit
zz = pd.DataFrame(X.toarray()[1:], columns=vectorizer_1.get_feature_names_out())
zz['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]
zz_2 = zz.copy()
zz_2.head() # only tf-idf
train_work_df = pd.merge(work_data, zz, on='contract_id') 

In [943]:
train_id
matrica_2 = scp.coo_matrix(
        (
            test_user_events_log['const'],  # оценки пользователей
            (test_user_events_log['contract_id'], test_user_events_log['name'])  # id пользователей и id аниме, для которых известны оценки
        ), 
        shape=(len(test_id), len(event_to_idx))  # размеры матрицы рейтингов
    ).todense().A
user_data = pd.DataFrame(data=matrica_2, columns=list(event_to_idx.keys()))
user_data['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]
unic = set()
st = ' '.join(["u"+str(i) for i in test_user_events_log.name.unique()])
res = [st]
for i in range(len(test_id)):
  user = test_user_events_log[test_user_events_log.contract_id == i]
  events = ' '.join(["u"+str(i) for i in list(sorted(user.name))])
  if len(events) == 0:
    events = '111'
  res.append(events)
  for i in events.split():
    unic.add(i)
X = vectorizer_2.transform(res) 
zz = pd.DataFrame(X.toarray()[1:], columns=vectorizer_2.get_feature_names_out())
zz['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]

In [944]:
work_data_2 = work_data.copy()
work_data_2.head()

,Добавление в Обращались с номеров,Обращение в службу заботы о клиентах,Выключение IPTV-пакета,Включение IPTV-пакета,"Включение услуги ""Ограничение функций ТВ""",Отключение услуги Аренда ТВ-приставки,Оборудование,Изменение истории,Продление действия тарифа,Предложение IPTV,Активация точки подключения на сутки,Смена тарифа,Удаление из Обращались с номеров,Смена пароля,Предложение ТВ,Внутреннее сообщение,Операция с mac-адресом,Закрепление сообщения,Перенос денежных средств,Изменение данных клиента,Возвращение клиента,Уведомления: подключение,Уведомления: подтверждение,Состояние клиентского оборудования,Смена схемы оплаты,"Включение услуги ""IPTV-Бонус""",Включена 20% скидка,Включение услуги Аренда ТВ-приставки,"Отключение услуги ""IPTV-Бонус""",Отключение скидки 20%,Предложение тарифа,Активация гарантированного платежа,Включение интернета на 20 минут,Изменение контактных телефонов,Внесение,Смена IP-адреса,Блокировка,Реестр договоров,Смена следующего тарифа,"Предложение акции ""Пригласи друга""",Предложение ТВ-пакета,Предложение автоплатежа,Активация ГП с IVR,Включение Бесплатной аренды,Замена оборудования,Удаление телефона из записи,Без границ бесплатно. Включение,Отключение скидки,Включена 30% скидка,Активация автоматического гарантированного платежа,Отключение услуги Автоплатёж,Расторжение,"Включена рассрочка за ""Wi-Fi маршрутизатор 2.4 ГГц - TP-LINK WR-850N или аналог""",Отмена гарантированного платежа,Манипуляции с оборудованием,Включена 10% скидка,Блокировка лицевого счета,Включение блокировки лицевого счета,Чек лист выполненных работ при регистрации,Разблокировка,Добавление IP-адреса,Удаление ip-адреса,Предложение расширенной гарантии на ТВ приставку,Без границ бесплатно. Отключение,Уведомления: отключение,Отключение скидки 10%,"Включение рассрочки за ""Подключение""",Окончание рассрочки за подключение,Предложение доп.гарантии на роутер,Предложение роутера,Тайный покупатель,Возвращение гарантированного платежа,"Включена рассрочка за ""IPTV-приставка""",Включение старых условий аренды,contract_id
0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1034
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1074
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1145
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1261
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1315


In [945]:
train_id
work_data_2 = work_data.copy()
is_ampty_work = np.zeros(len(test_id))
is_ampty_user = np.zeros(len(test_id))
count_call_work = np.zeros(len(test_id))
count_call_user = np.zeros(len(test_id))
for i in range(len(test_id)):
  # есть ли какая то инфа
  if idx_to_user[i] not in p1:
    is_ampty_work[i] = 1
  else:
    qq = test_events_log[test_events_log.contract_id == i]
    count_call_work[i] = qq.shape[0]
  if idx_to_user[i] not in p2:
    is_ampty_user[i] = 1
  else:
    qq = test_user_events_log[test_user_events_log.contract_id == i]
    count_call_user[i] = qq.shape[0]
work_data_2['is_ampty_work'] = is_ampty_work
work_data_2['count_call_work'] = count_call_work
user_data['is_ampty_user'] = is_ampty_user
user_data['count_call_user'] = count_call_user

first_event_work = np.zeros(len(test_id))  # первое событие
first_event_user = np.zeros(len(test_id))
last_event_work = np.zeros(len(test_id))  # последнее событие
last_event_user = np.zeros(len(test_id))
difference_events_work = np.zeros(len(test_id))  # разность первого и последнего
difference_events_user = np.zeros(len(test_id))
mean_events_work = np.zeros(len(test_id))  # среднее разность всех событий
mean_events_user = np.zeros(len(test_id))
for i in range(len(test_id)):
  # есть ли какая то инфа
  if idx_to_user[i] in p1:
    first_event_work[i], last_event_work[i], difference_events_work[i], mean_events_work[i] = find_out_the_time_work(i)
  if idx_to_user[i] in p2:
    first_event_user[i], last_event_user[i], difference_events_user[i], mean_events_user[i] = find_out_the_time_user(i)

In [946]:
big_df_last1 = pd.merge(user_data, work_data_2, on='contract_id')
big_df_last1 = pd.merge(big_df_last1, all_users, on='contract_id').drop_duplicates()
big_df_last1['first_event_work'] = first_event_work
big_df_last1['first_event_user'] = first_event_user
big_df_last1['last_event_work'] = last_event_work
big_df_last1['last_event_user'] = last_event_user
big_df_last1['difference_events_work'] = difference_events_work
big_df_last1['difference_events_user'] = difference_events_user
big_df_last1['mean_events_work'] = mean_events_work
big_df_last1['mean_events_user'] = mean_events_user
big_df_last2 = pd.merge(zz_2, zz, on='contract_id')
big_df_last = pd.merge(big_df_last1, big_df_last2, on='contract_id')
big_df_last = big_df_last.drop(["contract_id"], axis=1)

In [947]:
big_df_last.iloc[:5]

,Информер ВК. Показ,Включение интернета на 20 минут_x,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж_x,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение_x,Принято заявление и доп. соглашение по переезду,Смена пароля_x,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета_x,Выключение IPTV-пакета_x,Смена тарифа_x,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение_x,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,...,u16,u17,u18,u19,u2,u20,u21,u22,u23,u24,u25,u26,u27,u28,u29,u3,u30,u31,u32,u33,u34,u35,u36,u37,u38,u39,u4,u40,u41,u42,u43,u44,u45,u46,u47,u5,u6,u7,u8,u9
0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74567,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,5.0,5.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.952121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.613539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0


In [948]:
big_df_last['contract_id'] = [idx_to_user[i] for i in range(len(test_id))]
we_dont_know = big_df_last[(big_df_last.is_ampty_user == 1) & (big_df_last.is_ampty_work == 1)]
we_know = big_df_last[(big_df_last.is_ampty_user == 0) | (big_df_last.is_ampty_work == 0)]
we_know

,Информер ВК. Показ,Включение интернета на 20 минут_x,Показ информатора об изменении уведомлений через ВК платформу,Гарантированный платеж за деньги,Просмотр раздела Тайный покупатель,Включение интернета на 20 минут с IVR,Турбокнопка платно (1 час),Включение автоматической активации ГП,Отключение услуги Автоплатёж_x,Информер ВК. Закрытие,Виртуальное бессмертие. Включение,Виртуальное бессмертие. Использование,Без границ бесплатно. Отключение_x,Принято заявление и доп. соглашение по переезду,Смена пароля_x,Абонент оставил отзыв в Личном кабинете,Получение подарка в ЛК: сутки интернета на максимальной скорости,Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV,Включение IPTV-пакета_x,Выключение IPTV-пакета_x,Смена тарифа_x,Включение ограничения функций ТВ,Без границ платно. Отключение,Турбокнопка платно (1 сутки),Турбокнопка бесплатно,Без границ бесплатно. Включение_x,Без границ платно. Включение,Виртуальное бессмертие. Отключение,Тайный покупатель. Отправлена анкета,Бонус-скорость,Отключение ограничения функций ТВ,Изменение пакета HD. Закрытие лендинг-информера,Родительский контроль. Включение,Отключение управления услугами по паролю,Родительский контроль. Выключение,Восстановление пароля,Становление участником бонусной программы,Изменение схемы оплаты,Футболка Ситилинк,Отключение автоматической активации ГП,...,u17,u18,u19,u2,u20,u21,u22,u23,u24,u25,u26,u27,u28,u29,u3,u30,u31,u32,u33,u34,u35,u36,u37,u38,u39,u4,u40,u41,u42,u43,u44,u45,u46,u47,u5,u6,u7,u8,u9,contract_id
0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.745670,0.0,0.0,0.0,0.0,1034
2,5.0,5.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.952121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1145
3,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.613539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1261
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1315
6,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.228996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.659667,0.0,0.0,0.0,0.0,274468
296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0

In [949]:
len(idx_to_user), len(test_id)

(300, 300)

In [950]:
predictions_1 = model.predict(we_know.drop(['contract_id'], axis=1))
predictions_1

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0])

In [951]:
predictions_2 = np.zeros(len(model.predict(we_dont_know.drop(['contract_id'], axis=1))))
predictions_2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [952]:
we_know.contract_id
temp_1 = dict(zip(we_know.contract_id, predictions_1))
temp_2 = dict(zip(we_dont_know.contract_id, predictions_2))
x = {**temp_1, **temp_2}
result = pd.DataFrame({"contract_id": test_id,
                       "blocked": [x[i] for i in test_id]})

In [953]:
ttt = test_df.copy()
ttt['blocked'] = [int(x[i]) for i in ttt.contract_id]

In [954]:
ttt

,contract_id,blocked
0,1034,0
1,1074,0
2,1145,0
3,1261,0
4,1315,1
...,...,...
296,274468,1
297,274482,1
298,274486,1
299,274540,0


In [955]:
ttt.to_csv("submit.csv", index=False)

In [956]:
model.save_model("catboost.cbm")